In [32]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# 크롬 브라우저 열기
driver = webdriver.Chrome('C:/Temp/chromedriver')

driver.get("https://www.instagram.com/accounts/login/")
time.sleep(2)

email = '22milky@naver.com'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = '09Qhfkadl!' 
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

In [73]:
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url
import time

word = "산수갑산"
url = insta_searching(word)
driver.get(url)
time.sleep(5)

postnumber = driver.find_elements_by_css_selector("#react-root > section > main > header > div.WSpok > div > div:nth-child(2) > span > span")[0].text.replace(",","")
print(postnumber)

17905


In [38]:
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)
    
select_first(driver)

In [39]:
import re
from bs4 import BeautifulSoup

def get_content(driver):
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # 본문 내용 
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
    # 해시태그 
    tags = re.findall(r'#[^\s#,\\]+', content)  
    # 작성일자 
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # 좋아요 
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
    except:
        like = 0
    # 위치
    try: 
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
   
    data = [content, date, like, place, tags]
    instagram = [instagram, data]
    return data

get_content(driver)

['🐷산수갑산⠀✅이곳의 순대국밥은 꼭 한번 먹어봐야 한다고 봐~⠀⠀🧾순대정식(1.2)⠀⠀⠀✏️이곳의 순대정식은 다양한 부위로 구성된 순대모듬과 순대국의 국물이 함께 제공된다. 먼저 순대모듬은 대창순대, 오소리감투, 간, 머릿고기 등이 제공되어 탄탄한 구성이 돋보였고, 찹쌀이 들어간 대창순대의 고소함, 오소리감투의 쫄깃한 식감, 그리고 선도가 돋보여 부드러웠던 간이 가장 기억에 남았다. 뽀얀 빛깔의 순대국밥은 입안에 늘러 붙는 농밀한 국물에 진하고 묵직하기 보다 가벼움과 깔끔함이 강조되어 있어 만족스러웠다. 전국구로 이름을 날리고 있는 만큼 웨이팅은 감수해야 하고 국밥을 좋아한다면 꼭 한번 방문해봐야 할 성지와 같은 곳이다.⠀⠀⠀🍽 ★ ★ ★ ★ ☆⠀⠀⠀⠀📍서울 중구 을지로20길 24⠀⠀⠀#정슐랭 #정슐랭을지로 #정슐랭한식 #정슐랭산수갑산 #정슐랭별넷',
 '2021-08-16',
 0,
 '산수갑산',
 ['#정슐랭', '#정슐랭을지로', '#정슐랭한식', '#정슐랭산수갑산', '#정슐랭별넷']]

In [ ]:
def move_next(driver):

    right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

move_next(driver)

In [74]:
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url
import time

word = "산수갑산"
url = insta_searching(word)
driver.get(url)
postnumber = driver.find_elements_by_css_selector("span.g47SY")
time.sleep(5)

select_first(driver)
instagram = []

for i in range(int(postnumber) - 1) :
    get_content(driver)
    instagram = [instagram, data]
    move_next(driver)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [ ]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content','data','like','place','tags']
results_df.to_excel('C:/Users/POASISTOUR/Desktop/경북여행.xlsx')